In [1]:
import json
import gzip

import numpy as np
import pandas as pd
from sklearn import model_selection

pd.options.display.max_columns = 100
%matplotlib inline
#%load_ext lab_black

# Какие ошибки и лики мы можем найти?
Первый шаг к тому, чтобы определять лики в соревнованиях - это научиться находить ошибки в собственном и чужом коде

## Motorcycles

Прогнозируем стоимость мотоцикла и разбиваем данные на трейн и тест

In [ ]:
motorcycles = pd.read_csv(
    "https://gist.githubusercontent.com/aguschin/2b4fac72a9dc156fbb2ef7b05aadcf79/raw/8a8551de9fad8e220e588842dfd11641a96f8d68/MotorcyclesTSU.csv",
    parse_dates=["Published"],
)

In [ ]:
motorcycles.head()

In [ ]:
kf = model_selection.KFold(n_splits=2, shuffle=True)
for train_index, test_index in kf.split(motorcycles):
    print("TRAIN:", train_index, "TEST:", test_index)

## Github

Предсказываем взломы аккаунтов пользователей Github. Есть отдельная табличка `hacked_times` с идентифицированными взломами с колонками (`user_id`, `time_hacked`). Есть табличка с событиями `events`.

In [ ]:
hacked_times = pd.DataFrame(
    list(
        zip(
            np.random.randint(0, 100000000, 10),
            pd.Timestamp.now() + np.random.randn(10) * pd.Timedelta(days=1),
        )
    ),
    columns=["user_id", "time_hacked"],
)
hacked_times

In [ ]:
!wget https://data.gharchive.org/2015-01-01-15.json.gz

In [ ]:
with gzip.open("./2015-01-01-15.json.gz", "r") as fin:
    events = fin.read().decode("utf-8").split("\n")[:-1]
    events = [json.loads(e) for e in events]

In [ ]:
# actor -> id == user_id
events = pd.DataFrame(events)
events.head()

Варианты
1. Сделать KFold по табличке `hacked_times`, использовать её
2. Сделать KFold по табличке `events`, заполнить таргет из `hacked times`

## Shop sales

Предсказываем продажи магазина. Используем KFold для sales_train, обучаем модель, предсказываем для sales_test.

In [ ]:
sales_train = pd.read_csv(
    "https://gist.githubusercontent.com/aguschin/0498a6a6266933d2a10598b9dce2c4dd/raw/452d05b39c0f0289de9df85d172b0ed69885eb02/predict_future_sales_train_part.csv"
)

sales_test = pd.read_csv(
    "https://gist.githubusercontent.com/aguschin/18e83f0e7060f380ac2b59d2696e061b/raw/3d5f52ce4175182530f282cf0d888d07154a1521/predict_future_sales_test_part.csv"
)

In [ ]:
sales_train.head()

In [ ]:
sales_test.head()

## 

## Stock prices
Факт: предсказание стоимости акций - задача, где прогнозы слабо отличаются от случайных. Помимо этого, акции нередко испытывают особенно труднопрогнозируемые взлеты и падения. Часто бывает, что их наличие в обучающей выборке путает модель окончательно, и модели, которые обучались на отфильтрованной обучающей выборке, показывают себя более качественными при использовании в проде.

In [ ]:
stock_prices = pd.read_csv(
    "https://raw.githubusercontent.com/plotly/datasets/master/tesla-stock-price.csv"
)

In [ ]:
stock_prices["change"] = stock_prices["close"] - stock_prices["open"]

In [ ]:
stock_prices.change.hist(bins=100)

In [ ]:
((-20 < stock_prices["change"]) * (stock_prices["change"] < 20)).mean()

In [ ]:
stock_prices["change"] = stock_prices["change"].clip(-30, 30)

В чем проблема такого подхода и какие другие варианты можно предложить?